### Gerekli Kütüphaneler ve verilerin yüklenmesi

In [167]:
import pandas as pd 
import numpy as np 
import holidays
import datetime 

In [168]:
kesintiler = pd.read_csv('../Datasets/med.csv')
df = pd.read_csv('../Datasets/train.csv');df.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637
3,2018-01-01 03:00:00,1278.527266
4,2018-01-01 04:00:00,1220.697701


In [169]:
kesintiler['Kesinti'] = 1
kesintiler['Tarih'] = pd.to_datetime(kesintiler['Tarih'])
kesintiler.head(3)

,Tarih,Kesinti
0,2019-12-23,1
1,2019-06-12,1
2,2019-01-25,1


### Özellik Çıkarımı

Tarih kullanılarak gün, ay, yıl gibi özelliklerin çıkarılması ve kesintilerin veri setine dail edilmesi

In [170]:
def timestamp_extraction(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    
    df['Saat'] = df[date_col].dt.hour
    df['Aylık Gün'] = df[date_col].dt.day
    df['Yıllık Gün'] = df[date_col].dt.dayofyear
    df['Haftanın günü'] = df[date_col].dt.day_name()
    df['Hafta'] = df[date_col].dt.week
    df['Ay'] = df[date_col].dt.month
    df['Çeyreklik'] = df[date_col].dt.quarter
    df['Yıl'] = df[date_col].dt.year
    
    return df

df = timestamp_extraction(df, 'Tarih')

C:\Users\ramaz\AppData\Local\Temp\ipykernel_15368\3867835812.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['Hafta'] = df[date_col].dt.week


tatil günlerin belirlenmesi

In [171]:
tr_holidays = holidays.Turkey()
df['Özel Gün'] = df['Tarih'].apply(lambda x: x in tr_holidays);df.head()

,Tarih,Dağıtılan Enerji (MWh),Saat,Aylık Gün,Yıllık Gün,Haftanın günü,Hafta,Ay,Çeyreklik,Yıl,Özel Gün
0,2018-01-01 00:00:00,1593.944216,0,1,1,Monday,1,1,1,2018,True
1,2018-01-01 01:00:00,1513.933887,1,1,1,Monday,1,1,1,2018,True
2,2018-01-01 02:00:00,1402.612637,2,1,1,Monday,1,1,1,2018,True
3,2018-01-01 03:00:00,1278.527266,3,1,1,Monday,1,1,1,2018,True
4,2018-01-01 04:00:00,1220.697701,4,1,1,Monday,1,1,1,2018,True


In [172]:
len(df[df['Özel Gün']==True]) / len(df[df['Özel Gün']==False])

0.04236760124610592

mevsimlerin belirlenmesi

In [173]:
def get_season(month):
    if month >= 3 and month <= 5:
        return 'Spring'
    elif month >= 6 and month <= 8:
        return 'Summer'
    elif month >= 9 and month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

In [174]:
df['Mevsim'] = df['Tarih'].dt.month.apply(get_season);df.head()

,Tarih,Dağıtılan Enerji (MWh),Saat,Aylık Gün,Yıllık Gün,Haftanın günü,Hafta,Ay,Çeyreklik,Yıl,Özel Gün,Mevsim
0,2018-01-01 00:00:00,1593.944216,0,1,1,Monday,1,1,1,2018,True,Winter
1,2018-01-01 01:00:00,1513.933887,1,1,1,Monday,1,1,1,2018,True,Winter
2,2018-01-01 02:00:00,1402.612637,2,1,1,Monday,1,1,1,2018,True,Winter
3,2018-01-01 03:00:00,1278.527266,3,1,1,Monday,1,1,1,2018,True,Winter
4,2018-01-01 04:00:00,1220.697701,4,1,1,Monday,1,1,1,2018,True,Winter


Hava durumu özelliğinin eklenmesi

In [175]:
weather = pd.read_csv('../Web Scraping/weather.csv')
weather.head()

,date,weather_info
0,2018-01-01,Min/max4°/12°c
1,2018-01-02,Min/max7°/14°c
2,2018-01-03,Min/max9°/14°c
3,2018-01-04,Min/max8°/13°c
4,2018-01-05,Min/max7°/12°c


In [176]:
weather.isnull().sum()

date            0
weather_info    0
dtype: int64

In [177]:
weather[['unit', 'min', 'max']] = weather['weather_info'].str.extract(r'(Min/max)(-?\d+)°/(-?\d+)°c')

In [178]:
weather.head()

,date,weather_info,unit,min,max
0,2018-01-01,Min/max4°/12°c,Min/max,4,12
1,2018-01-02,Min/max7°/14°c,Min/max,7,14
2,2018-01-03,Min/max9°/14°c,Min/max,9,14
3,2018-01-04,Min/max8°/13°c,Min/max,8,13
4,2018-01-05,Min/max7°/12°c,Min/max,7,12


In [179]:
weather.isnull().sum()

date            0
weather_info    0
unit            0
min             0
max             0
dtype: int64

In [180]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1704 non-null   object
 1   weather_info  1704 non-null   object
 2   unit          1704 non-null   object
 3   min           1704 non-null   object
 4   max           1704 non-null   object
dtypes: object(5)
memory usage: 66.7+ KB


In [181]:
# set min/max col to integer
weather['min'] = weather['min'].astype(int)
weather['max'] = weather['max'].astype(int)

In [182]:
weather.drop(columns=['weather_info', 'unit'], inplace=True)
weather.head()

,date,min,max
0,2018-01-01,4,12
1,2018-01-02,7,14
2,2018-01-03,9,14
3,2018-01-04,8,13
4,2018-01-05,7,12


In [183]:
weather.columns = ['date', 'Min_temperature', 'Max_temperature']

In [184]:
weather.to_csv('../Datasets/weather.csv', index=False)

In [185]:
weather = pd.read_csv('../Datasets/weather.csv')

In [186]:
weather.head()

,date,Min_temperature,Max_temperature
0,2018-01-01,4,12
1,2018-01-02,7,14
2,2018-01-03,9,14
3,2018-01-04,8,13
4,2018-01-05,7,12


In [187]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             1704 non-null   object
 1   Min_temperature  1704 non-null   int64 
 2   Max_temperature  1704 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 40.1+ KB


In [188]:
weather['date'] = pd.to_datetime(weather['date']).dt.date

In [189]:
df['tarih_no_time'] = df['Tarih'].dt.date

In [190]:
df = pd.merge(df, weather, left_on='tarih_no_time', right_on='date', how='left');df.head()

,Tarih,Dağıtılan Enerji (MWh),Saat,Aylık Gün,Yıllık Gün,Haftanın günü,Hafta,Ay,Çeyreklik,Yıl,Özel Gün,Mevsim,tarih_no_time,date,Min_temperature,Max_temperature
0,2018-01-01 00:00:00,1593.944216,0,1,1,Monday,1,1,1,2018,True,Winter,2018-01-01,2018-01-01,4,12
1,2018-01-01 01:00:00,1513.933887,1,1,1,Monday,1,1,1,2018,True,Winter,2018-01-01,2018-01-01,4,12
2,2018-01-01 02:00:00,1402.612637,2,1,1,Monday,1,1,1,2018,True,Winter,2018-01-01,2018-01-01,4,12
3,2018-01-01 03:00:00,1278.527266,3,1,1,Monday,1,1,1,2018,True,Winter,2018-01-01,2018-01-01,4,12
4,2018-01-01 04:00:00,1220.697701,4,1,1,Monday,1,1,1,2018,True,Winter,2018-01-01,2018-01-01,4,12


In [191]:
df.drop(['tarih_no_time', 'date'], axis=1, inplace=True);df.head()

,Tarih,Dağıtılan Enerji (MWh),Saat,Aylık Gün,Yıllık Gün,Haftanın günü,Hafta,Ay,Çeyreklik,Yıl,Özel Gün,Mevsim,Min_temperature,Max_temperature
0,2018-01-01 00:00:00,1593.944216,0,1,1,Monday,1,1,1,2018,True,Winter,4,12
1,2018-01-01 01:00:00,1513.933887,1,1,1,Monday,1,1,1,2018,True,Winter,4,12
2,2018-01-01 02:00:00,1402.612637,2,1,1,Monday,1,1,1,2018,True,Winter,4,12
3,2018-01-01 03:00:00,1278.527266,3,1,1,Monday,1,1,1,2018,True,Winter,4,12
4,2018-01-01 04:00:00,1220.697701,4,1,1,Monday,1,1,1,2018,True,Winter,4,12


In [192]:
def is_weekend(date_str):
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
    return 1 if date_obj.weekday() >= 5 else 0

In [194]:
df['Hafta Sonu'] = df['Tarih'].dt.strftime('%Y-%m-%d').apply(is_weekend);df.tail()

,Tarih,Dağıtılan Enerji (MWh),Saat,Aylık Gün,Yıllık Gün,Haftanın günü,Hafta,Ay,Çeyreklik,Yıl,Özel Gün,Mevsim,Min_temperature,Max_temperature,Hafta Sonu
40147,2022-07-31 19:00:00,2492.489498,19,31,212,Sunday,30,7,3,2022,False,Summer,22,38,1
40148,2022-07-31 20:00:00,2479.958245,20,31,212,Sunday,30,7,3,2022,False,Summer,22,38,1
40149,2022-07-31 21:00:00,2485.016884,21,31,212,Sunday,30,7,3,2022,False,Summer,22,38,1
40150,2022-07-31 22:00:00,2396.574255,22,31,212,Sunday,30,7,3,2022,False,Summer,22,38,1
40151,2022-07-31 23:00:00,2285.252844,23,31,212,Sunday,30,7,3,2022,False,Summer,22,38,1


<hr>